### Experiments for Section 4.1: Memorization

In [ ]:
import numpy as np
import keras
import tensorflow as tf
import os
import json
import pickle
from collections import defaultdict, namedtuple
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from utils import models
from utils import datasets
from utils import entropy_estimators as ee

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

## MLP

Hyperparameters

In [ ]:
cfg = {}
cfg['dataset'] = 'MNIST'
cfg['model'] = 'MLP'
cfg['width'] = [1024,1024,1024,1024]
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 100
cfg['patience'] = 20

cfg['dropout'] = []
cfg['noise_ratio'] = 0.4  # change the label noise ratio here
cfg['batch_norm'] = []
cfg['weight_decay'] = []

In [ ]:
run = 1

arch =  '-'.join(map(str,cfg['width']))
exp_name = 'label_noise/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
if cfg['noise_ratio'] > 0:
    exp_name += '_LabelNoise_'+str(cfg['noise_ratio'])
    
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

In [ ]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [ ]:
trn, tst = datasets.get_dataset(cfg)

In [ ]:
with open(cfg['exp_name']+'/noisy_trn', 'wb') as f:
    pickle.dump(trn._asdict(), f, pickle.HIGHEST_PROTOCOL)
with open(cfg['exp_name']+'/tst', 'wb') as f:
    pickle.dump(tst._asdict(), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
model = models.get_model(cfg, trn)
model.summary()

In [ ]:
if not os.path.exists(cfg['exp_name']+'/saved_models'):
    print("Making directory", cfg['exp_name']+'/saved_models')
    os.makedirs(cfg['exp_name']+'/saved_models')
model.save(cfg['exp_name']+"/saved_models/model-00")

In [ ]:
lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)
ckpt = ModelCheckpoint(cfg['exp_name']+"/saved_models/model-{epoch:02d}", verbose=1, save_best_only=False)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              validation_data=(tst.X, tst.Y),
              callbacks = [lr_decay, ckpt])

In [ ]:
epochs_list = [0,2,4,6,8,10,20,30,40,50,60,70,80,90,100]

In [ ]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []
for epoch in epochs_list:
    model = keras.models.load_model(cfg['exp_name']+"/saved_models/model-%02d"%epoch)
    
    if cfg['noise_ratio'] > 0:
        Dataset = namedtuple('Dataset',['X','Y','y'])
        with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
            trn = Dataset(**pickle.load(f))
        with open(cfg['exp_name']+'/tst', 'rb') as f:
            tst = Dataset(**pickle.load(f))
    
    trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)
    train_loss.append(trn_results[0])
    train_acc.append(trn_results[1])
    tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=0)
    test_loss.append(tst_results[0])
    test_acc.append(tst_results[1])

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
smi_all_epochs = defaultdict(list)

for epoch in epochs_list:
    filepath = cfg['exp_name']+"/saved_models/model-%02d"%epoch
    model = keras.models.load_model(filepath)
    
    if cfg['noise_ratio'] > 0:
        Dataset = namedtuple('Dataset',['X','Y','y'])
        with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
            trn = Dataset(**pickle.load(f))
    noisy_label = trn.Y.argmax(1)
    
    smi_all_layers = []
    print('Epoch:',epoch)
    count = 0
    for l,layer in enumerate(model.layers):
        if hasattr(layer, 'kernel'):
            count +=1
            int_model = keras.Model(inputs=model.inputs, outputs=model.layers[l].output)
            activity = int_model.predict(trn.X[:10000])
            smi = ee.compute_smi(x=activity, y=noisy_label[:10000], m=500)
            smi_all_layers.append(smi)
            print('Layer %d: SI(T;Y) = %0.3f'%(count,smi))
    smi_all_epochs['Epoch%02d'%epoch] = smi_all_layers

fname = cfg['exp_name']+'/smi'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi_all_epochs, f, pickle.HIGHEST_PROTOCOL)

## CNN

In [ ]:
cfg = {}
cfg['dataset'] = 'Fashion_MNIST'
cfg['model'] = 'CNN_Global'
cfg['width'] = [512,512,512,512]
cfg['optimizer'] = 'SGD'
cfg['learning_rate'] = 0.01
cfg['batch_size']    = 32
cfg['n_epochs'] = 100
cfg['n_train'] = None

cfg['dropout'] = []
cfg['weight_decay'] = []
cfg['batch_norm'] = [True,True,True,True]
cfg['noise_ratio'] = 0

In [ ]:
run = 1

arch =  '-'.join(map(str,cfg['width']))
exp_name = 'label_noise/'+cfg['model']+'_'+arch+'_'+cfg['dataset']+'_'+cfg['optimizer']
if len(cfg['batch_norm']) > 0:
    bn =  '-'.join(map(str,cfg['batch_norm']))
    exp_name += '_BatchNorm_'+bn
if cfg['noise_ratio'] > 0:
    exp_name += '_LabelNoise_'+str(cfg['noise_ratio'])
    
cfg['exp_name'] = exp_name + '/run_%d'%(run)
if not os.path.exists(cfg['exp_name']):
    print("Making directory", cfg['exp_name'])
    os.makedirs(cfg['exp_name'])

In [ ]:
fname = cfg['exp_name'] + '/config.json'
with open(fname, 'w') as f:
    json.dump(cfg, f)

In [ ]:
trn, tst = datasets.get_dataset(cfg)

if cfg['noise_ratio'] > 0:
    with open(cfg['exp_name']+'/noisy_trn', 'wb') as f:
        pickle.dump(trn._asdict(), f, pickle.HIGHEST_PROTOCOL)
    with open(cfg['exp_name']+'/tst', 'wb') as f:
        pickle.dump(tst._asdict(), f, pickle.HIGHEST_PROTOCOL)

In [ ]:
model = models.get_model(cfg, trn)
model.summary()

In [ ]:
if not os.path.exists(cfg['exp_name']+'/saved_models'):
    print("Making directory", cfg['exp_name']+'/saved_models')
    os.makedirs(cfg['exp_name']+'/saved_models')
model.save(cfg['exp_name']+"/saved_models/model-00")

In [ ]:
def lr_step_decay(epoch, lr):
    drop_rate = 0.9
    epochs_drop = 20
    return cfg['learning_rate'] * np.power(drop_rate, np.floor(epoch/epochs_drop))

lr_decay = ReduceLROnPlateau(monitor='accuracy', factor=0.9, patience=10, verbose=1)
ckpt = ModelCheckpoint(cfg['exp_name']+"/saved_models/model-{epoch:02d}", verbose=1, save_best_only=False)

r = model.fit(x=trn.X, y=trn.Y, 
              verbose    = 1, 
              batch_size = cfg['batch_size'],
              epochs = cfg['n_epochs'],
              callbacks = [lr_decay, ckpt])

In [ ]:
epochs_list = [0,2,4,6,8,10,20,30,40,50,60,70,80,90,100]

In [ ]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []
for epoch in epochs_list:
    model = keras.models.load_model(cfg['exp_name']+"/saved_models/model-%02d"%epoch)
    
    if cfg['noise_ratio'] > 0:
        Dataset = namedtuple('Dataset',['X','Y','y'])
        with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
            trn = Dataset(**pickle.load(f))
        with open(cfg['exp_name']+'/tst', 'rb') as f:
            tst = Dataset(**pickle.load(f))
    
    trn_results = model.evaluate(trn.X, trn.Y, batch_size=32, verbose=0)
    train_loss.append(trn_results[0])
    train_acc.append(trn_results[1])
    tst_results = model.evaluate(tst.X, tst.Y, batch_size=32, verbose=0)
    test_loss.append(tst_results[0])
    test_acc.append(tst_results[1])

metrics={}
metrics['train_loss'] = train_loss
metrics['train_acc'] = train_acc
metrics['test_loss'] = test_loss
metrics['test_acc'] = test_acc

fname = cfg['exp_name']+'/metrics'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(metrics, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
smi_all_epochs = defaultdict(list)

for epoch in epochs_list:
    filepath = cfg['exp_name']+"/saved_models/model-%02d"%epoch
    model = keras.models.load_model(filepath)
    
    if cfg['noise_ratio'] > 0:
        Dataset = namedtuple('Dataset',['X','Y','y'])
        with open(cfg['exp_name']+'/noisy_trn', 'rb') as f:
            trn = Dataset(**pickle.load(f))
            
    noisy_label = trn.Y.argmax(1)
    
    smi_all_layers = []
    print('Epoch:',epoch)
    count = 0
    for l,layer in enumerate(model.layers):
        if isinstance(layer, keras.layers.Activation) or isinstance(layer, keras.layers.GlobalAveragePooling2D):
            count +=1
            int_model = keras.Model(inputs=model.inputs, outputs=model.layers[l].output)
            activity = int_model.predict(trn.X[:10000], batch_size=32)
            if len(activity.shape) > 2:
                activity = activity.reshape(activity.shape[0],-1)
            smi = ee.compute_smi(x=activity, y=noisy_label[:10000], m=500)
            smi_all_layers.append(smi)
            print('Layer %d: SI(T;Y) = %0.3f'%(count,smi))

        
    smi_all_epochs['Epoch%02d'%epoch] = smi_all_layers

fname = cfg['exp_name']+'/smi_corrupted'
print("Saving", fname)
with open(fname, 'wb') as f:
    pickle.dump(smi_all_epochs, f, pickle.HIGHEST_PROTOCOL)